# DuckDB 
In this notebook, we will demonstrate how to use DuckDB, for accessing and querying data efficiently. DuckDB is designed to work seamlessly with modern analytical workloads, making it a powerful tool for data analysis, research, and question-answering systems.

To begin, ensure you have DuckDB installed in your Python environment. You can easily install it using `pip install duckdb`. DuckDB does not require a server, so you can start querying data directly in your local environment without any additional setup.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/Demo/duckdb.ipynb)

In [ ]:
!pip install indox
!pip install duckdb
!pip install semantic_text_splitter
!pip install sentence-transformers

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indox
```
2. **Activate the virtual environment:**
```bash
indox_judge\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
```

2. **Activate the virtual environment:**
    ```bash
   source indox/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


### Load Hugging face API key 

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']

In [6]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


Initialize a language model and an embedding model using the indox library with Hugging Face and Azure services. The HuggingFaceModel class is used to create an instance of the Mistral-7B-Instruct model for tasks like question answering, while the AzureEmbedding would handle embedding tasks.

In [7]:
from indox.llms import HuggingFaceModel
from indox.embeddings import AzureOpenAIEmbeddings
mistral_qa = HuggingFaceModel(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
azure_embed = AzureOpenAIEmbeddings(api_key=OPENAI_API_KEY,model="text-embedding-3-small")

INFO: Initializing HuggingFaceModel with model: mistralai/Mistral-7B-Instruct-v0.2
INFO: HuggingFaceModel initialized successfully
INFO: Initialized OpenAiEmbedding with model: text-embedding-3-small


### Load Sample text 

In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

In [9]:
file_path = "sample.txt"
with open(file_path, "r") as file:
    text = file.read()

use the `RecursiveCharacterTextSplitter` class from the indox library to divide a large text into smaller, manageable chunks

In [10]:
from indox.splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(400,20)
content_chunks = splitter.split_text(text)

In [12]:
content_chunks[:3]

["The wife of a rich man fell sick, and as she felt that her end\n\nwas drawing near, she called her only daughter to her bedside and\n\nsaid, dear child, be good and pious, and then the\n\ngood God will always protect you, and I will look down on you\n\nfrom heaven and be near you.  Thereupon she closed her eyes and\n\ndeparted.  Every day the maiden went out to her mother's grave,",
 'and wept, and she remained pious and good.  When winter came\n\nthe snow spread a white sheet over the grave, and by the time the\n\nspring sun had drawn it off again, the man had taken another wife.\n\nThe woman had brought with her into the house two daughters,\n\nwho were beautiful and fair of face, but vile and black of heart.\n\nNow began a bad time for the poor step-child.  Is the stupid goose',
 'to sit in the parlor with us, they said.  He who wants to eat bread\n\nmust earn it.  Out with the kitchen-wench.  They took her pretty\n\nclothes away from her, put an old grey bedgown on her, and gave\

### Set up vector store
Set up a vector store using the `DuckDB` class from the indox library.

In [15]:
from indox.vector_stores import DuckDB
vector_store = DuckDB(
    embedding_function=azure_embed,
    vector_key="embedding",   
    id_key="id",              
    text_key="text",          
    table_name="embeddings"
)

### Storing Data in the Vector Store

In [ ]:
vector_store.add(texts=content_chunks)

In [17]:
query = "How cinderella reach her happy ending?"


In [18]:
retriever = indox.QuestionAnswer(vector_database=vector_store,llm=mistral_qa,top_k=5)


In [20]:
answer = retriever.invoke(query=query)


INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: text-embedding-3-small


2024-09-02 19:18:21,773 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Sending request to Hugging Face API
INFO: Received successful response from Hugging Face API
INFO: Query answered successfully


In [21]:
answer

"Cinderella reached her happy ending when the prince recognized her at the royal ball, and they got married. She received her golden dress and glass slippers from her fairy godmother, and went to the ball incognito. Her step-mother and stepsisters didn't recognize her and believed she was a foreign princess."

In [23]:
context = retriever.context
context[:2]

["which they had wished for, and to cinderella he gave the branch\n\nfrom the hazel-bush.  Cinderella thanked him, went to her mother's\n\ngrave and planted the branch on it, and wept so much that the tears\n\nfell down on it and watered it.  And it grew and became a handsome\n\ntree. Thrice a day cinderella went and sat beneath it, and wept and\n\nprayed, and a little white bird always came on the tree, and if",
 "glove.  And when she rose up and the king's son looked at her\n\nface he recognized the beautiful maiden who had danced with\n\nhim and cried, that is the true bride.  The step-mother and\n\nthe two sisters were horrified and became pale with rage, he,\n\nhowever, took cinderella on his horse and rode away with her.  As\n\nthey passed by the hazel-tree, the two white doves cried -"]